### Lab 2: Hyponyms and Hypernyms

In [1]:
import pandas as pd
import os
import sys
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json
num_cores = multiprocessing.cpu_count()
print(num_cores)
%matplotlib inline

4


In [2]:
df = pd.read_csv('nlp2.csv', sep = '|', encoding='utf-8')
df.head()

,HYPONYM,HYPERNYM,TEXT,LINK
0,ПАПЕРТЬ,внешний притвор,Паперть — непокрытая кровлей площадка перед вн...,https://ru.wikipedia.org/wiki/Паперть\t\t
1,ПАПЕРТЬ,площадка перед церковью,Паперть — площадка перед церковью и крыльцо.,http://window.edu.ru/catalog/pdf2txt/568/25568...
2,ПАПЕРТЬ,церковное крыльцо,"Паперть — церковное крыльцо, площадка перед вх...",https://slovar.cc/rus/ushakov/426566.html\t\t
3,ПАПЕРТЬ,крытая площадка,Паперть — крытая площадка перед входом в церковь.,https://how-to-all.com/значение:паперть\t\t
4,ПАПЕРТЬ,закрытая площадка,Паперть — закрытая площадка перед входом в цер...,http://drevne-rus-lit.niv.ru/drevne-rus-lit/te...


In [3]:
def prestr(x):
    return str(x).replace('\"','').replace("'",'"')

In [4]:
class DefDict(defaultdict):
    def __missing__(self, key):
        self[key] = key
        return key
    
idx2syns = DefDict(lambda x:x)
for val in df.values:
    idx2syns[val[0]]=val[1]
    try:
        pidxs = json.loads(prestr(val[2]))
        concp = [el.split(",")[0] for el in json.loads(prestr(val[2]))]
        idx2syns.update(dict(zip(pidxs,concp)))
    except:
        print(prestr(val[1]))
        print(prestr(val[2]))

внешний притвор
Паперть — непокрытая кровлей площадка перед внутренним притвором храма, на которой в первые века христианства стояли плачущие и кающиеся. 
площадка перед церковью
Паперть — площадка перед церковью и крыльцо.
церковное крыльцо
Паперть — церковное крыльцо, площадка перед входом в церковь.
крытая площадка
Паперть — крытая площадка перед входом в церковь.
закрытая площадка
Паперть — закрытая площадка перед входом в церковь .
род пистолета
Парабеллум — род автоматического скорострельного пистолета.
самозарядный пистолет
Парабеллум — автоматический самозарядный пистолет Георга Люгера, более известный под названием «Парабеллум» (от лат. Para bellum — готовься к войне), стал легендарным немецким оружием начала XX века.
оружие
Парабеллум — германский пистолет, о котором слышали многие, оружие, по праву ставшее символом немецкого пистолета первой половины двадцатого века. 
пистолет
Парабеллум — пистолет, разработанный в 1900 году Георгом Люгером для вооружения германской армии.
о

In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
p = morph.parse('какой-либо')[0]
p.tag.POS

'ADJF'

In [6]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [7]:
from yargy import rule, or_

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN'),
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')

INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),
)

INCLUDING_O = or_(
    eq('в'),
    eq('широком'),
    eq('смысле'),
    eq('слова')
)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
    )

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или')
        ),
        eq('другие')
    ),
    
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('PREP'),
        gram('ADJF'),
        gram('NPRO')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('NPRO'),
    ),
    rule(
        gram('NOUN'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN')
    ),
    
)


Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [8]:
def get_hyperonym(i):
    main_word = (df['HYPONYM'][i]).lower()
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(START, MID, END),
        rule(HYPONYM, INCLUDING_O, MID, END),
        rule(INCLUDING_O, MID, END)
)
    parser = Parser(RULE) 
    text = df['TEXT'][i]
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    hyperonyms = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyperonyms.append(k)
        print(k)
    print('-----------------------------------------')
    print(' ')

In [9]:
for i in range(len(df)):
    get_hyperonym(i)

Паперть — непокрытая кровлей площадка перед внутренним притвором храма, на которой в первые века христианства стояли плачущие и кающиеся. 
['паперть', '—', 'непокрытая', 'кровлей', 'площадка', 'перед', 'внутренним', 'притвором', 'храма']
-----------------------------------------
 
Паперть — площадка перед церковью и крыльцо.
['паперть', '—', 'площадка', 'перед', 'церковью', 'и', 'крыльцо']
-----------------------------------------
 
Паперть — церковное крыльцо, площадка перед входом в церковь.
['паперть', '—', 'церковное', 'крыльцо']
-----------------------------------------
 
Паперть — крытая площадка перед входом в церковь.
['паперть', '—', 'крытая', 'площадка', 'перед', 'входом', 'в', 'церковь']
-----------------------------------------
 
Паперть — закрытая площадка перед входом в церковь .
['паперть', '—', 'закрытая', 'площадка', 'перед', 'входом', 'в', 'церковь']
-----------------------------------------
 
Парабеллум — род автоматического скорострельного пистолета.
['парабеллум', 

['парео', '—', 'это', 'квадратный', 'или', 'прямоугольный', 'отрез']
-----------------------------------------
 
На сегодняшний день парео считается самым универсальным — изобретением моды. 
-----------------------------------------
 
Парео — это самый удобный наряд для пляжа.
['парео', '—', 'это', 'самый', 'удобный', 'наряд']
-----------------------------------------
 
ПАРКОВЩИК — человек, занимающийся паркованием автомобилей на автостоянке, расставляющий их по местам
['парковщик', '—', 'человек']
-----------------------------------------
 
ПАРКОВЩИК — это работник автостоянки.
['парковщик', '—', 'это', 'работник', 'автостоянки']
-----------------------------------------
 
ПАРКОВЩИК — работник парковочной стоянки, организующий парковку гостевых автомобилей на специально отведенной территории. 
['парковщик', '—', 'работник', 'парковочной', 'стоянки']
-----------------------------------------
 
Парковщик относится к категории служащих и непосредственно подчиняется.
---------------------

 
Универсальные клещи, объединяющие в себе плоскогубцы, кусачки, отвертку и т.п.
-----------------------------------------
 
Универсальные клещи, объединяющие в себе плоскогубцы, кусачки, отвертку и т.п.
-----------------------------------------
 
Универсальные клещи, объединяющие в себе плоскогубцы, кусачки, отвертку и т.п.
-----------------------------------------
 
ПАССАТИЖИ — комбинированные щипцы, объединяющие в себе плоскогубцы, кусачки, отвертку и другие инструменты.
['пассатижи', '—', 'комбинированные', 'щипцы']
['отвертку', 'и', 'другие', 'инструменты']
-----------------------------------------
 
Пассатижи — многофункциональный ручной слесарно-монтажный инструмент, предназначенный для зажима и захвата труб и деталей разных форм.
-----------------------------------------
 
Пастила — кондитерское изделие из фруктовой массы и сахара, обычно с добавлением яичных белков.
['пастила', '—', 'кондитерское', 'изделие', 'из', 'фруктовой', 'массы', 'и', 'сахара']
-------------------------

пеленгатор — Прибор для пеленгации, для определения местонахождения какого-л. объекта.
['пеленгатор', '—', 'прибор', 'для', 'пеленгации']
-----------------------------------------
 
Пеленгатор — прибор, установленный на компасе, служащий для взятия пеленгов или направлений на земные предметы или небесные светила. Старое название — алидада.
['пеленгатор', '—', 'прибор']
['старое', 'название', '—', 'алидада']
-----------------------------------------
 
ПЕЛЕНГАТОР — особый визир, помещаемый на крышке котелка компаса и служащий для определения компасных направлений (компасных пеленгов) на различные видимые с судна предметы. 
['пеленгатор', '—', 'особый', 'визир']
-----------------------------------------
 
ПЕЛЕНГАТОР — Угломерный инструмент, состоящий из круга, разделенного на градусы, с двумя подвижными мишенями, служить вспомогателем при пеленговании.
['пеленгатор', '—', 'угломерный', 'инструмент']
-----------------------------------------
 


In [10]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
p = morph.parse('Паразитология')[0]
p.tag.POS

'NOUN'